In [1]:
import pandas as pd
import difflib
import numpy as np
import re

In [2]:
data = pd.read_csv('/Users/christophermccarthy/Documents/datathon_2021/Purchase_Card_Transactions.csv')
ranges = pd.read_csv('/Users/christophermccarthy/Documents/datathon_2021/iso18245_ranges.csv')
mcc_codes = pd.read_csv('/Users/christophermccarthy/Documents/datathon_2021/visa_list.csv')

In [ ]:
# GET GROUPS

ranges.index = pd.IntervalIndex.from_arrays(ranges['MCC START'],ranges['MCC END'],closed='both')
mcc_codes['GROUP'] = mcc_codes['MCC'].apply(lambda x : ranges.iloc[ranges.index.get_loc(x)]['DESCRIPTION']
                                           )

In [ ]:
#Lower Case
def strip_punc(x):
    x = str(x)
    x = x.replace("&", 'and')
    x = re.sub(r'[^\w\s]','',x)
    x = x.replace(" ", '')
    x = x.replace("shop", 'store')
    return x

mcc_codes['DESCRIPTION'] = mcc_codes.DESCRIPTION.str.lower()
data['MCC DESCRIPTION'] = data.DESCRIPTION.str.lower()

mcc_codes['DESCRIPTION'] = mcc_codes.DESCRIPTION.apply(lambda x : strip_punc(x))
data['MCC DESCRIPTION'] = data.DESCRIPTION.(lambda x : strip_punc(x))

mcc_codes = mcc_codes.set_index('DESCRIPTION')


In [ ]:
# Get close matches
def fix_shitty_format(s):
    ret_dict = {}
    for code in s:
        close_matches = difflib.get_close_matches(code, mcc_codes.index)
        if len(close_matches) != 0:
            ret_dict[code] = close_matches[0]
        else:
            ret_dict[code] = ''
    return ret_dict

fix_dict = fix_shitty_format(pd.Series(data.DESCRIPTION.unique()))
data['join_key'] == data.DESCRIPTION.apply(lambda x : fix_dict[x])

In [ ]:
data_fixed = data.join(mcc_codes, on = 'join_key')